# Install Dependencies


In [1]:
!curl -Hx-api-key:"7wjJOwaiWM73wSrEHQljH69ZyAJ5VG2376XzAwRH" https://api.semanticscholar.org/

<!doctype html>
<html lang=en>
<title>Redirecting...</title>
<h1>Redirecting...</h1>
<p>You should be redirected automatically to the target URL: <a href="https://www.semanticscholar.org/product/api">https://www.semanticscholar.org/product/api</a>. If not, click the link.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100   273  100   273    0     0    161      0  0:00:01  0:00:01 --:--:--   161


In [2]:
import json
import openai
from decouple import config
import requests

In [3]:
OPEN_API_KEY = config('OPENAI_API_KEY')
openai.api_key = OPEN_API_KEY

# Prompt ChatGPT for prompts to query databases with

In [4]:
question = "How can we design a vehicle to travel from Sydney to USA in one hour?"

messages = []
messages.append({'role': 'user', 
'content': f"""You are BIDARA, a biomimetic designer and research assistant, and a leading expert in biomimicry, biology, engineering, industrial design, environmental science, physiology, and paleontology. Focus on understanding, learning from, and emulating the strategies used by living things, with the intention of creating designs and technologies that are sustainable.\n\n" \
"Your goal is to help the user work in a step by step way through the Biomimicry Design Process to propose biomimetic solutions to a challenge. Cite peer reviewed sources for your information.\n\n" \
I want you to provide a query to an academic database for each of the biomimetic solutions you've developed. \
Provide the search query in a JSON in the following format. \
{{"queries": {"biological_idea1:query1"}, {"biological_idea2:query2"}}} \
"1. Biologize - Analyze the essential functions and context your design challenge must address. Reframe them in biological terms, so that you can “ask nature” for advice. The goal of this step is to arrive at one or more “How does nature…?” questions that can guide your research as you look for biological models in the next step. To broaden the range of potential solutions, turn your question(s) around and consider opposite, or tangential functions. For example, if your biologized question is “How does nature retain liquids?”, you could also ask “How does nature repel liquids?” because similar mechanisms could be at work in both scenarios (i.e. controlling the movement of a liquid). Or if you are interested in silent flight and you know that flight noise is a consequence of turbulence, you might also ask how nature reduces turbulence in water, because air and water share similar fluid dynamics.\n" \
"2. Discover - Look for natural models (organisms and ecosystems) that need to address the same functions and context as your design solution. Identify the strategies used that support their survival and success. This step focuses on research and information gathering. You want to generate as many possible sources for inspiration as you can, using your “how does nature…” questions (from the Biologize step) as a guide. Look across multiple species, ecosystems, and scales and learn everything you can about the varied ways that nature has adapted to the functions and contexts relevant to your challenge.\n" \
"3. Abstract - Carefully study the essential features or mechanisms that make the biological strategy successful. Write a design strategy that describes how the features work to meet the function(s) you’re interested in in great detail. Try to come up with discipline-neutral synonyms for any biological terms (e.g. replace “fur” with “fibers,” or “skin” with “membrane”) while staying true to the science. The design strategy should clearly address the function(s) you want to meet within the context it will be used. It is not a statement about your design or solution; it’s a launching pad for brainstorming possible solutions. Stay true to the biology. Don’t jump to conclusions about what your design will be; just capture the strategy so that you can stay open to possibilities. When you are done, review your design strategy with a critical eye. Have you included all of the pertinent information? Does your design strategy capture the lesson from nature that drew you to the biological strategy in the first place? Does it give you new insights or simply validate existing design approaches?\n\n" \
"Here’s a simply stated biological strategy:\n" \
"The polar bear’s fur has an external layer of hollow, translucent (not white) guard hairs that transmit heat from sunlight to warm the bear’s skin, while a dense underfur prevents the warmth from radiating back out.\n\n" \
"A designer might be able to brainstorm design solutions using just that. But more often, in order to actually create a design based on what we can learn from biology, it helps to remove biological terms and restate it in design language.\n\n" \
"Here’s a design strategy based on the same biological strategy:\n" \
"A covering keeps heat inside by having many translucent tubes that transmit heat from sunlight to warm the inner surface, while next to the inner surface, a dense covering of smaller diameter fibers prevents warmth from radiating back out.\n\n" \
"Stating the strategy this way makes it easier to translate it into a design application. (An even more detailed design strategy might talk about the length of the fibers or the number of fibers per square centimeter, e.g., if that information is important and its analog can be found in the biological literature.)") \
"\n\n{question}" \
"""})


response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0,
        )


In [5]:
messages

[{'role': 'user',
  'content': 'You are BIDARA, a biomimetic designer and research assistant, and a leading expert in biomimicry, biology, engineering, industrial design, environmental science, physiology, and paleontology. Focus on understanding, learning from, and emulating the strategies used by living things, with the intention of creating designs and technologies that are sustainable.\n\n" "Your goal is to help the user work in a step by step way through the Biomimicry Design Process to propose biomimetic solutions to a challenge. Cite peer reviewed sources for your information.\n\n" I want you to provide a query to an academic database for each of the biological ideas you\'ve developed. Provide the search query in a JSON in the following format. {{"queries": {"biological_idea1:query1"}, {"biological_idea2:query2"}} "1. Biologize - Analyze the essential functions and context your design challenge must address. Reframe them in biological terms, so that you can “ask nature” for advi

In [6]:
response

<OpenAIObject chat.completion id=chatcmpl-7jY2jJAKyMChEAjGyrJwXH51TUVsk at 0x2604e0c7b50> JSON: {
  "id": "chatcmpl-7jY2jJAKyMChEAjGyrJwXH51TUVsk",
  "object": "chat.completion",
  "created": 1691089977,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\"4. Emulate - Translate the design strategy into a design concept. This is where you brainstorm possible solutions that use the design strategy you\u2019ve developed. You might come up with a single design concept or several. The goal is to create a design that meets the function(s) you\u2019re interested in, within the context it will be used. You might need to iterate between the Abstract and Emulate steps as you learn more about the design constraints and opportunities. \n\n\" \"5. Evaluate - Test your design concept(s) to see if they meet the function(s) and context you\u2019re interested in. This might involve building a prototype, running simulation

In [10]:
print(messages[0]['content'])

You are BIDARA, a biomimetic designer and research assistant, and a leading expert in biomimicry, biology, engineering, industrial design, environmental science, physiology, and paleontology. Focus on understanding, learning from, and emulating the strategies used by living things, with the intention of creating designs and technologies that are sustainable.

" "Your goal is to help the user work in a step by step way through the Biomimicry Design Process to propose biomimetic solutions to a challenge. Cite peer reviewed sources for your information.

" I want you to provide a query to an academic database for each of the biological ideas you've developed. Provide the search query in a JSON in the following format. {{"queries": {"biological_idea1:query1"}, {"biological_idea2:query2"}} "1. Biologize - Analyze the essential functions and context your design challenge must address. Reframe them in biological terms, so that you can “ask nature” for advice. The goal of this step is to arriv

In [11]:
print(response['choices'][0]['message']['content'])

"4. Emulate - Translate the design strategy into a design concept. This is where you brainstorm possible solutions that use the design strategy you’ve developed. You might come up with a single design concept or several. The goal is to create a design that meets the function(s) you’re interested in, within the context it will be used. You might need to iterate between the Abstract and Emulate steps as you learn more about the design constraints and opportunities. 

" "5. Evaluate - Test your design concept(s) to see if they meet the function(s) and context you’re interested in. This might involve building a prototype, running simulations, or doing a life cycle assessment. You might need to iterate between the Emulate and Evaluate steps as you refine your design concept. 

" "6. Communicate - Share your design concept and the biological strategy that inspired it. This might involve writing a report, giving a presentation, or creating a video. The goal is to help others understand how yo

In [8]:
jsonstring = response['choices'][0]['message']['content']
jsonstring = "" + jsonstring

In [9]:
query_list = json.loads(jsonstring)
query_list['queries']

['Design principles for hypersonic vehicles',
 'Advancements in hypersonic travel',
 'Challenges in transcontinental hypersonic travel',
 'Propulsion systems for hypersonic vehicles',
 'Materials for high-speed aerospace vehicles',
 'Fuel efficiency in hypersonic vehicles',
 'Safety measures in hypersonic travel']

In [29]:
results = requests.get('https://api.semanticscholar.org/graph/v1/paper/search?query=covid+vaccination&offset=100&limit=3')

In [30]:
json_results = json.loads(results.text)

In [31]:
json_results

{'total': 1170604,
 'offset': 100,
 'next': 103,
 'data': [{'paperId': '15241c09b59f6b1bbfa69867a29b554dff0cd04c',
   'title': 'Willingness to vaccinate against COVID-19 among Bangladeshi adults: Understanding the strategies to optimize vaccination coverage'},
  {'paperId': 'edd135e5dc5dbfe3f54c7d9f8be08706b06c779f',
   'title': 'Healthcare workers’ (HCWs) attitudes and related factors towards COVID-19 vaccination: a rapid systematic review'},
  {'paperId': '73326621034f67beab8a6834345270b0379e47ec',
   'title': 'Myocarditis and Pericarditis After Vaccination for COVID-19.'}]}

In [32]:
for paper in json_results['data']:
    print(paper['title'])

Willingness to vaccinate against COVID-19 among Bangladeshi adults: Understanding the strategies to optimize vaccination coverage
Healthcare workers’ (HCWs) attitudes and related factors towards COVID-19 vaccination: a rapid systematic review
Myocarditis and Pericarditis After Vaccination for COVID-19.


In [33]:
def stringToAPIformat(string1):
    string1 = string1.replace(" ", "+")
    return string1
assert stringToAPIformat("Hello there") == "Hello+there"

In [34]:
number_of_papers_to_retrieve = 3
data_to_retrieve = "title" # could be "abstract", "title"
# look into passing paper, getting summary 


for query in query_list['queries']:
    URLtoGET = f'https://api.semanticscholar.org/graph/v1/paper/search?query={stringToAPIformat(query)}&limit={number_of_papers_to_retrieve}&fields={data_to_retrieve}'
    GETresult = requests.get(URLtoGET)
    json_GETresult = json.loads(GETresult.text)
    print(f"Top {number_of_papers_to_retrieve} {data_to_retrieve}s for query: '{query}'")
    if json_GETresult['total'] == 0:
        print("No papers found \n")
        continue
    for paper in json_GETresult['data']:
        print("\t" + paper[data_to_retrieve])
    print("\n")
    

Top 3 titles for query: 'Design principles for hypersonic vehicles'
	Design and numerical simulation of a clamshell-shaped inlet cover for air-breathing hypersonic vehicles
	Design and numerical simulation of a clamshell-shaped inlet cover for air-breathing hypersonic vehicles
	Design of Dual-Mode Engine Flowpaths for Hypersonic Vehicles Using Reduced-Order Models


Top 3 titles for query: 'Advancements in hypersonic travel'
	Numerical study of a stepped aerospike design at various Mach numbers
	COMPRESSIBLE FLOW SIMULATIONS FOR SHOCK TUBE PERFORMANCE ANALYSIS


Top 3 titles for query: 'Challenges in transcontinental hypersonic travel'
No papers found 

Top 3 titles for query: 'Propulsion systems for hypersonic vehicles'
	Supersonic Combustion in Air-Breathing Propulsion Systems for Hypersonic Flight
	Supersonic Combustion in Air-Breathing Propulsion Systems for Hypersonic Flight
	HYPERSONIC PROPULSION SYSTEMS – A REVIEW OF DESIGN SOLUTIONS


Top 3 titles for query: 'Materials for high